In [1]:
import pyautogui as gui
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cap =cv2.VideoCapture(0)
#for motion
lower_yellow = np.array([10,120,125])   #real yellow
upper_yellow = np.array([35,255,255])
 #for left click
lower_green= np.array([55,39,7])   #real green
upper_green = np.array([112,160,136])
 # for right click
lower_purple = np.array([130,90,80])  #real purple
upper_purple = np.array([150,180,180])

while cap.isOpened():
    ret,frame = cap.read()
    frame = cv2.flip(frame,1)

    #smoothen the Image
    image_smooth = cv2.GaussianBlur(frame,(7,7),0)
    
    # Define ROI
    mask = np.zeros_like(frame)
    mask[50:350,50:350]=[255,255,255]
    image_roi = cv2.bitwise_and(image_smooth,mask)
    cv2.rectangle(frame, (50,50), (350,350), (0,0,255), 2)
    cv2.line(frame, (150,50), (150,350), (0,0,255), 1)
    cv2.line(frame, (250,50), (250,350), (0,0,255), 1)
    cv2.line(frame, (50,150), (350,150), (0,0,255), 1)
    cv2.line(frame, (50,250), (350,250), (0,0,255), 1)
    
    # threshold the image for yellow color
    image_hsv = cv2.cvtColor(image_roi,cv2.COLOR_BGR2HSV)
    image_threshold = cv2.inRange(image_hsv,lower_yellow,upper_yellow)
    
    # finding conotur 
    contours,hierarchy = cv2.findContours(image_threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    # find index of the largest contour (using argmax)
    if (len(contours)!= 0):
        areas = [cv2.contourArea(c) for c in contours]
        max_index = np.argmax(areas)
        cnt = contours[max_index]
        
        # pointer on video
        M = cv2.moments(cnt)
        if (M["m00"]!=0):
            cx=int(M["m10"]/M["m00"])
            cy=int(M["m01"]/M["m00"])
            cv2.circle(frame,(cx,cy),4,(0,255,0),-1)
            
            # cursor motion
            if cx < 150:
                dist_x = -20
            elif cx > 250:
                dist_x = 20
            else:
                dist_x = 0
                
            if cy < 150:
                dist_y = -20
            elif cy > 250:
                dist_y = 20
            else:
                dist_y = 0
            gui.moveRel(dist_x,dist_y,duration=0)    
            
    # check for left click
    image_threshold_green = cv2.inRange(image_hsv,lower_green,upper_green)
    contour_green,hierarchy = cv2.findContours(image_threshold_green,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    if(len(contour_green)!=0):
        gui.click()
        cv2.waitKey(100)
            
    # check for right click
    image_threshold_purple = cv2.inRange(image_hsv,lower_purple,upper_purple)
    contour_purple,hierarchy = cv2.findContours(image_threshold_purple,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    if(len(contour_purple)!=0):
        gui.click(button='right')
        cv2.waitKey(100)
            
    cv2.imshow("object tracking",frame)
    cv2.imshow("object tracking yellow",image_threshold)
    cv2.imshow("object tracking green",image_threshold_green)
    cv2.imshow("object tracking purple",image_threshold_purple)
    key = cv2.waitKey(10)
    if key == ord("q"):
        break


cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 